In [2]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import re, glob
import os, sys
from scipy import stats

# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from config import clientID, clientSEC


In [ ]:
# Spotify API info
client_credentials_manager = SpotifyClientCredentials(client_id=clientID,client_secret=clientSEC)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Time Setup  (will loop later)
starting_month = 4
ending_month = 4
starting_year = 2019
ending_year = 2021
years = ending_year - starting_year

# Define country with an ISO code
country = "IT" # Italy

"""
['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'SV', 'EE', 'FI'
, 'FR','DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE', 'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 'MY', 'MT', 'MX', 'MC'
, 'NL', 'NZ','NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'ES', 'SK', 'SE', 'CH', 'TW', 'TR', 'GB', 'US', 'UY']
"""